In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

use_cuda = torch.cuda.is_available()

normalize = transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

preprocess_train = transforms.Compose([transforms.RandomResizedCrop(256),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    normalize])

preprocess_test = transforms.Compose([transforms.RandomResizedCrop(256),
                                    transforms.ToTensor(),
                                    normalize])

train_data = datasets.ImageFolder(root='dogImages/train',transform=preprocess_train)
test_data = datasets.ImageFolder(root='dogImages/test',transform=preprocess_test)
valid_data = datasets.ImageFolder(root='dogImages/valid',transform=preprocess_test)

train_data_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=32,shuffle=True,num_workers=1)
test_data_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=32,shuffle=True,num_workers=1)
valid_data_loader = torch.utils.data.DataLoader(dataset=valid_data,batch_size=32,shuffle=True,num_workers=1)

loaders_scratch = {
        "train" : train_data_loader,
        "test" : test_data_loader,
        "valid" : valid_data_loader,
}

#printing and check the loaded data
print(train_data_loader)
print("train_data size", len(train_data))
print("test_data size", len(test_data))
print("valid_data size", len(valid_data))

train_data size 6680
test_data size 836
valid_data size 835


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv1=nn.Conv2d(3,16,4,stride=1)
        self.conv2=nn.Conv2d(16,32,4,stride=1)
        self.conv3=nn.Conv2d(32,64,4,stride=1)
        self.conv4=nn.Conv2d(64,128,4,stride=1)
        self.conv5=nn.Conv2d(128,256,4,stride=1)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(256*5*5,1024)
        self.fc2=nn.Linear(1024,512)
        self.fc3=nn.Linear(512,133)
        self.dropout=nn.Dropout(0.25)
    
    def forward(self, x):
        ## Define forward behavior
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(F.relu(self.conv3(x)))
        x=self.pool(F.relu(self.conv4(x)))
        x=self.pool(F.relu(self.conv5(x)))
        x=x.view(-1,256*5*5)
        x=self.dropout(x)
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        x=F.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.fc3(x)
        return x
    
# instantiate the CNN
model_scratch = Net()
    
use_cuda = torch.cuda.is_available()

if use_cuda:
    model_scratch.cuda()

criterion_scratch = nn.CrossEntropyLoss()
### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.025)
#empty the cache
torch.cuda.empty_cache()

In [3]:

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
        """returns trained model"""
        # initialize tracker for minimum validation loss
        valid_loss_min = np.Inf 
        for epoch in range(1, n_epochs+1):
            # initialize variables to monitor training and validation loss
            train_loss = 0.0
            valid_loss = 0.0
            ###################
            # train the model #
            ###################
            model.train()
            for batch_idx, (data, target) in enumerate(loaders['train']):
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                ## find the loss and update the model parameters accordingly
                ## record the average training loss, using something like
                # clear the gradients of all optimized variables
                optimizer.zero_grad()
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model.forward(data)
                # calculate the batch loss
                loss = criterion(output, target)
                # backward pass: compute gradient of the loss with respect to model parameters
                loss.backward()
                # perform a single optimization step (parameter update)
                optimizer.step()
                # update training loss
                train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            ######################    
            # validate the model #
            ###################### 
            model.eval()
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                ## update the average validation loss
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model.forward(data)
                # calculate the batch loss
                loss = criterion(output, target)
                # update validation loss
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            # print training/validation statistics 
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss
                ))
            ## TODO: save the model if validation loss has decreased
            # Save model if validation loss has decreased since last min
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
                torch.save(model.state_dict(), save_path)
                valid_loss_min = valid_loss
        # return trained model
        return model
    
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
    
    # train the model
model_scratch = train(35, loaders_scratch, model_scratch, optimizer_scratch, 
                                    criterion_scratch, use_cuda, 'model_scratch.pt')
    # load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))
#Here is my outputs from training:

OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Anaconda3\envs\DLND\lib\site-packages\torch\utils\data\_utils\worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Anaconda3\envs\DLND\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Anaconda3\envs\DLND\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Anaconda3\envs\DLND\lib\site-packages\torchvision\datasets\folder.py", line 138, in __getitem__
    sample = self.loader(path)
  File "C:\Anaconda3\envs\DLND\lib\site-packages\torchvision\datasets\folder.py", line 174, in default_loader
    return pil_loader(path)
  File "C:\Anaconda3\envs\DLND\lib\site-packages\torchvision\datasets\folder.py", line 157, in pil_loader
    return img.convert('RGB')
  File

In [ ]:
def test(loaders, model, criterion, use_cuda):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
            100. * correct / total, correct, total))
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)